# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.5.0


# Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model (class version)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

# Create model (function version)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.4350, train_loss: 2.15953064, train_accuracy: 0.3203, test_Accuracy: 0.2599
Epoch: [ 0] [    1/  468] time: 1.7890, train_loss: 2.09979773, train_accuracy: 0.4922, test_Accuracy: 0.4327
Epoch: [ 0] [    2/  468] time: 2.2510, train_loss: 2.03756332, train_accuracy: 0.5703, test_Accuracy: 0.5634
Epoch: [ 0] [    3/  468] time: 2.6603, train_loss: 1.93012381, train_accuracy: 0.6328, test_Accuracy: 0.6242
Epoch: [ 0] [    4/  468] time: 2.9900, train_loss: 1.83192825, train_accuracy: 0.6719, test_Accuracy: 0.6580
Epoch: [ 0] [    5/  468] time: 3.2660, train_loss: 1.75624657, train_accuracy: 0.6641, test_Accuracy: 0.6825
Epoch: [ 0] [    6/  468] time: 3.5271, train_loss: 1.64536357, train_accuracy: 0.6953, test_Accuracy: 0.6960
Epoch: [ 0] [    7/  468] time: 3.7960, train_loss: 1.54017603, train_accuracy: 0.6562, test_Accuracy: 0.7175
Epoch: [ 0] [    8/  468] time: 4.0720,

Epoch: [ 0] [   74/  468] time: 24.8883, train_loss: 0.27514941, train_accuracy: 0.8984, test_Accuracy: 0.9019
Epoch: [ 0] [   75/  468] time: 25.2753, train_loss: 0.28075945, train_accuracy: 0.9219, test_Accuracy: 0.8985
Epoch: [ 0] [   76/  468] time: 25.5648, train_loss: 0.30172557, train_accuracy: 0.9375, test_Accuracy: 0.8994
Epoch: [ 0] [   77/  468] time: 25.9903, train_loss: 0.36016193, train_accuracy: 0.9375, test_Accuracy: 0.9039
Epoch: [ 0] [   78/  468] time: 26.3123, train_loss: 0.34255910, train_accuracy: 0.9062, test_Accuracy: 0.9104
Epoch: [ 0] [   79/  468] time: 26.6843, train_loss: 0.21832234, train_accuracy: 0.9141, test_Accuracy: 0.9142
Epoch: [ 0] [   80/  468] time: 27.0045, train_loss: 0.30616730, train_accuracy: 0.9219, test_Accuracy: 0.9145
Epoch: [ 0] [   81/  468] time: 27.3803, train_loss: 0.22338063, train_accuracy: 0.9453, test_Accuracy: 0.9106
Epoch: [ 0] [   82/  468] time: 27.7395, train_loss: 0.22152111, train_accuracy: 0.9453, test_Accuracy: 0.9077
E

Epoch: [ 0] [  148/  468] time: 48.0083, train_loss: 0.23679218, train_accuracy: 0.9375, test_Accuracy: 0.9265
Epoch: [ 0] [  149/  468] time: 48.2974, train_loss: 0.23255673, train_accuracy: 0.9219, test_Accuracy: 0.9271
Epoch: [ 0] [  150/  468] time: 48.5983, train_loss: 0.23130690, train_accuracy: 0.9375, test_Accuracy: 0.9238
Epoch: [ 0] [  151/  468] time: 48.9013, train_loss: 0.33634630, train_accuracy: 0.9062, test_Accuracy: 0.9229
Epoch: [ 0] [  152/  468] time: 49.2033, train_loss: 0.21502949, train_accuracy: 0.9219, test_Accuracy: 0.9227
Epoch: [ 0] [  153/  468] time: 49.5133, train_loss: 0.31194353, train_accuracy: 0.9219, test_Accuracy: 0.9246
Epoch: [ 0] [  154/  468] time: 49.8604, train_loss: 0.26622906, train_accuracy: 0.9453, test_Accuracy: 0.9270
Epoch: [ 0] [  155/  468] time: 50.1933, train_loss: 0.42380330, train_accuracy: 0.8906, test_Accuracy: 0.9289
Epoch: [ 0] [  156/  468] time: 50.5233, train_loss: 0.18533973, train_accuracy: 0.9453, test_Accuracy: 0.9307
E

Epoch: [ 0] [  222/  468] time: 70.9963, train_loss: 0.21087360, train_accuracy: 0.9219, test_Accuracy: 0.9416
Epoch: [ 0] [  223/  468] time: 71.5573, train_loss: 0.15379122, train_accuracy: 0.9688, test_Accuracy: 0.9399
Epoch: [ 0] [  224/  468] time: 72.0653, train_loss: 0.18060891, train_accuracy: 0.9375, test_Accuracy: 0.9412
Epoch: [ 0] [  225/  468] time: 72.3693, train_loss: 0.19603488, train_accuracy: 0.9375, test_Accuracy: 0.9416
Epoch: [ 0] [  226/  468] time: 72.6723, train_loss: 0.14300090, train_accuracy: 0.9766, test_Accuracy: 0.9421
Epoch: [ 0] [  227/  468] time: 73.0413, train_loss: 0.13290335, train_accuracy: 0.9609, test_Accuracy: 0.9413
Epoch: [ 0] [  228/  468] time: 73.3803, train_loss: 0.13129002, train_accuracy: 0.9531, test_Accuracy: 0.9396
Epoch: [ 0] [  229/  468] time: 73.7273, train_loss: 0.26217058, train_accuracy: 0.9219, test_Accuracy: 0.9358
Epoch: [ 0] [  230/  468] time: 74.0404, train_loss: 0.36450684, train_accuracy: 0.8906, test_Accuracy: 0.9369
E

Epoch: [ 0] [  296/  468] time: 95.3763, train_loss: 0.21648544, train_accuracy: 0.9297, test_Accuracy: 0.9468
Epoch: [ 0] [  297/  468] time: 95.6683, train_loss: 0.21035275, train_accuracy: 0.9219, test_Accuracy: 0.9474
Epoch: [ 0] [  298/  468] time: 96.0033, train_loss: 0.09398817, train_accuracy: 0.9609, test_Accuracy: 0.9475
Epoch: [ 0] [  299/  468] time: 96.3174, train_loss: 0.29431981, train_accuracy: 0.8984, test_Accuracy: 0.9468
Epoch: [ 0] [  300/  468] time: 96.6843, train_loss: 0.11575428, train_accuracy: 0.9688, test_Accuracy: 0.9459
Epoch: [ 0] [  301/  468] time: 97.0173, train_loss: 0.12681091, train_accuracy: 0.9531, test_Accuracy: 0.9447
Epoch: [ 0] [  302/  468] time: 97.3043, train_loss: 0.24450563, train_accuracy: 0.9297, test_Accuracy: 0.9456
Epoch: [ 0] [  303/  468] time: 97.5833, train_loss: 0.28451520, train_accuracy: 0.9219, test_Accuracy: 0.9465
Epoch: [ 0] [  304/  468] time: 97.9113, train_loss: 0.07665053, train_accuracy: 0.9922, test_Accuracy: 0.9479
E

Epoch: [ 0] [  370/  468] time: 117.9573, train_loss: 0.14653903, train_accuracy: 0.9375, test_Accuracy: 0.9556
Epoch: [ 0] [  371/  468] time: 118.2284, train_loss: 0.10365078, train_accuracy: 0.9688, test_Accuracy: 0.9559
Epoch: [ 0] [  372/  468] time: 118.5013, train_loss: 0.09957595, train_accuracy: 0.9609, test_Accuracy: 0.9551
Epoch: [ 0] [  373/  468] time: 118.7843, train_loss: 0.09007403, train_accuracy: 0.9688, test_Accuracy: 0.9547
Epoch: [ 0] [  374/  468] time: 119.0783, train_loss: 0.15016595, train_accuracy: 0.9609, test_Accuracy: 0.9542
Epoch: [ 0] [  375/  468] time: 119.3763, train_loss: 0.10878884, train_accuracy: 0.9609, test_Accuracy: 0.9537
Epoch: [ 0] [  376/  468] time: 119.6743, train_loss: 0.12009349, train_accuracy: 0.9766, test_Accuracy: 0.9533
Epoch: [ 0] [  377/  468] time: 119.9633, train_loss: 0.17167749, train_accuracy: 0.9375, test_Accuracy: 0.9531
Epoch: [ 0] [  378/  468] time: 120.2523, train_loss: 0.13206078, train_accuracy: 0.9766, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 141.5373, train_loss: 0.06303954, train_accuracy: 0.9844, test_Accuracy: 0.9578
Epoch: [ 0] [  445/  468] time: 141.7933, train_loss: 0.11336780, train_accuracy: 0.9609, test_Accuracy: 0.9572
Epoch: [ 0] [  446/  468] time: 142.1253, train_loss: 0.19061215, train_accuracy: 0.9453, test_Accuracy: 0.9569
Epoch: [ 0] [  447/  468] time: 142.4963, train_loss: 0.08623877, train_accuracy: 0.9688, test_Accuracy: 0.9594
Epoch: [ 0] [  448/  468] time: 142.8673, train_loss: 0.13827649, train_accuracy: 0.9766, test_Accuracy: 0.9597
Epoch: [ 0] [  449/  468] time: 143.3803, train_loss: 0.08980080, train_accuracy: 0.9453, test_Accuracy: 0.9597
Epoch: [ 0] [  450/  468] time: 143.7536, train_loss: 0.08793849, train_accuracy: 0.9844, test_Accuracy: 0.9589
Epoch: [ 0] [  451/  468] time: 144.0813, train_loss: 0.08917171, train_accuracy: 0.9688, test_Accuracy: 0.9578
Epoch: [ 0] [  452/  468] time: 144.5803, train_loss: 0.08612387, train_accuracy: 0.9766, test_Accuracy:

# Test accuracy : 95.89 %